In [30]:
import pandas as pd
import pandas_profiling
import numpy as np
import json
from pymystem3 import Mystem
from urllib.request import urlopen
from pandas.io.json import json_normalize
import collections
import ast

In [3]:
COUNT = 214802000

In [4]:
def process_question(question):
    data = {key: question.get(key) for key in ('bestanswer', 'arating', 'points', 'nick', 'totalMarks', 'qtext', 'added', 'rating', 'lvl', 'qid', 'anscnt', 'is_expert', 'usrid', 'kpd' )} 
    data['tags'] = question['category'].get('name')
    return data

In [5]:
import socket
import urllib.request

timeout = 3
socket.setdefaulttimeout(timeout)

#get all questions
all_questions = {}
for i in range(14348178, 14348178+33408971):
    with open('questions.jsonl', 'a') as outfile:
        try:
            req = urllib.request.Request('https://otvet.mail.ru/api/v2/question?qid={}'.format(i))
            questions_response = urllib.request.urlopen(req)
            question_result = json.loads(questions_response.read().decode('utf-8'))
            json.dump(process_question(question_result), outfile)
            outfile.write('\n')
        except Exception as exception:
            pass
#             print(i)
#             print(exception)

KeyboardInterrupt: 

In [46]:
questions_df = pd.DataFrame()
questions = []
with open('questions3.jsonl', 'r') as f:
    for question in f:
        json_normalized_question = json_normalize(json.loads(question))
        questions.append(json_normalized_question)
questions_df = questions_df.append(questions)

In [47]:
questions_df.to_csv(index=False, path_or_buf='mailru.csv')

In [42]:
questions_df = pd.read_csv('mailru2.csv')

In [9]:
questions_df.head(20)

,added,anscnt,arating,bestanswer,is_expert,kpd,lvl,nick,points,qid,qtext,rating,tags,totalMarks,usrid
0,270897190,11,0,277566917,0,5.49,Профи,Виталий Райзер,852,47757150,"Любите ли Вы ПИВО?Если да,то сколько можете вы...",0,Напитки,NaN,22011279
1,270217004,1,7,277575677,0,0,Ученик,Татьяна,199,47757151,сколько по времени заниматься на этом массажор...,1,Занятия спортом,NaN,12213995
2,270217052,3,7,277565364,0,7.31,Знаток,GENKA,265,47757152,у меня иж ода полный привод. кто знает от чего...,0,"Сервис, Обслуживание, Тюнинг",NaN,10169457
3,270938802,2,0,277566207,0,10.05,Ученик,Клостридия У клостридии нет,12,47757153,"Будет ли поджелудочная выделять свои ферменты,...",0,Прочее о здоровье и красоте,NaN,32144973
4,270935187,11,0,277574598,0,17.54,Искусственный Интеллект,Полина Подольская,115934,47757154,А вы ДО СЛЕЗ давно смеялись!!!?)))))))))))от Д...,0,Отношения,NaN,31201572
5,270608419,1,2,0,0,12.87,Знаток,Анна Иванова,362,47757155,Где можно скачать/посмотреть эффектные клипы к...,0,Музыка,NaN,32186092
6,129129955,2,3,277565003,0,42.85,Ученик,Максим Казанцев,162,47757156,В свое время слышал запись концерта в память г...,0,"Концерты, Выставки, Спектакли",NaN,24418158
7,270608420,7,4,0,0,5.78,Гуру,максим тоскин,3322,47757157,"СТРАННИКи - это те ,кто без паспорта ?.",0,Общество,NaN,22462408
8,270216352,1,6,277565005,0,0,Ученик,суд_без_истины_байдарка_без_вёсел_,4,47757158,"магнитный компас показывал, что этот указывает...",1,"Архитектура, Скульптура",NaN,35249538
9,270216823,1,3,277564751,0,0,Ученик,7u76u 67i67ri,85,47757159,перегородка из газоблоков толщин 10 см. можно ...,0,Техника,NaN,35249932


In [39]:
top_tags = ['Транспорт', 'Бизнес и финансы', 'Другое', 'Животные и растения', 'Еда и кулинария', 'Искусство и культура', 'Компьютерные и видеоигры', 'Образование', 'Спорт', 'Семья и отношения', 'Развлечения и досуг', 'Работа и карьера', 'Путешествия', 'Помощь с учёбой', 'Общество и история и политика', 'Мода и стиль', 'Красота и здоровье', 'Компьютеры и интернет']

def tags_mapper(service_name, tag):
    if service_name == 'znatoki':
        if tag == 'Авто и транспорт':
            return 'Транспорт'
        if tag == 'Бизнес и право':
            return 'Бизнес и финансы'
        if tag == 'Животные и домашние питомцы':
            return 'Животные и растения'
        if tag == 'Еда и кулинария':
            return 'Еда и кулинария'
        if tag == 'Искусство и культура':
            return 'Искусство и культура'
        if tag == 'Компьютерные и видеоигры':
            return 'Компьютерные и видеоигры'
        if tag == 'Компьютеры, электроника, интернет':
            return 'Компьютеры и интернет'
        if tag == 'Красота и здоровье':
            return 'Красота и здоровье'
        if tag == 'Мода и стиль':
            return 'Мода и стиль'
        if tag == 'Общество, история, политика':
            return 'Общество и история'
        if tag == 'Помощь с учёбой':
            return 'Помощь с учёбой'
        if tag == 'Путешествия и география':
            return 'Путешествия'
        if tag == 'Работа и карьера':
            return 'Работа и карьера'
        if tag == 'Растения и садоводство' or tag == 'Развлечения и досуг':
            return 'Развлечения и досуг'
        if tag == 'Семья и отношения':
            return 'Семья и отношения'
        if tag == 'Спорт':
            return 'Спорт'
        if tag == 'Точные и естественные науки' or tag == 'Языки и гуманитарные науки':
            return 'Образование'
        if tag == 'Дом и быт':
            return 'Другое'
    elif service_name == 'mailru':
        if tag == 'Авто, Мото':
            return 'Транспорт'
        if tag == 'Бизнес, Финансы':
            return 'Бизнес и финансы'
        if tag == 'Животные, Растения':
            return 'Животные и растения'
        if tag == 'Еда и кулинария':
            return 'Еда и кулинария'
        if tag == 'Искусство и культура':
            return 'Искусство и культура'
        if tag == 'Компьютерные и Видео игры':
            return 'Компьютерные и видеоигры'
        if tag == 'Компьютеры, Связь':
            return 'Компьютеры и интернет'
        if tag == 'Красота и Здоровье':
            return 'Красота и здоровье'
        if tag == 'Стиль, Мода, Звезды':
            return 'Мода и стиль'
        if tag == 'Общество, Политика, СМИ':
            return 'Общество и история'
        if tag == 'Домашние задания':
            return 'Помощь с учёбой'
        if tag == 'Города и Страны' or tag == 'Путешествия, Туризм':
            return 'Путешествия'
        if tag == 'Программирование' or tag == 'Работа, Карьера' or tag == 'Юридическая консультация':
            return 'Работа и карьера'
        if tag == 'Досуг, Развлечения' or tag == 'Юмор' or tag == 'Фотография, Видеосъемка' or tag == 'Гороскопы, Магия, Гадания':
            return 'Развлечения и досуг'
        if tag == 'Знакомства, Любовь, Отношения' or tag == 'Семья, Дом, Дети':
            return 'Семья и отношения'
        if tag == 'Спорт':
            return 'Спорт'
        if tag == 'Образование':
            return 'Образование'
        if tag == 'Товары и Услуги' or tag =='Философия, Непознанное':
            return 'Другое'
    else:
        if tag == 'Метрополитен' or tag == 'Авиация' or tag == 'Авто' or tag == 'Транспорт':
            return 'Транспорт'
        if tag == 'Деньги' or tag == 'Банки' or tag == 'Бизнес' or tag == 'Финансы':
            return 'Бизнес и финансы'
        if tag == 'Животные' or tag == 'Природа':
            return 'Животные и растения'
        if tag == 'Кафе и рестораны' or tag == 'Еда':
            return 'Еда и кулинария'
        if tag == 'Культура' or tag == 'Искусство' or tag == 'Искусство и культура':
            return 'Искусство и культура'
        if tag == 'YouTube' or tag == 'Игры' or tag == 'Компьютерные игры':
            return 'Компьютерные и видеоигры'
        if tag == 'Медиа' or tag == 'Видео' or tag == 'IT (информационные технологии)' or tag == 'Apple' or tag == 'Гаджеты' or tag == 'Безопасность' or tag == 'Интернет' or tag == 'Техника' or tag == 'Технологии' or tag == 'ВКонтакте' or tag == 'Реклама' or tag == 'Новости' or tag == 'Социальные сети' or tag == 'Компьютеры':
            return 'Компьютеры и интернет'
        if tag == 'Смерть' or tag == 'Наркотики' or tag == 'Волосы' or tag == 'Алкоголь' or tag == 'Болезнь' or tag == 'Внешность' or tag == 'здоровое питание' or tag == 'Здоровье' or tag == 'Красота' or tag == 'Курение' or tag == 'Медицина' or tag == 'Организм' or tag == 'Питание' or tag == 'Человек' or tag == 'Тело' or tag == 'Физиология':
            return 'Красота и здоровье'
        if tag == 'Дизайн' or tag == 'Знаменитости' or tag == 'Шоппинг' or tag == 'Мода' or tag == 'Стиль' or tag == 'Одежда':
            return 'Мода и стиль'
        if tag == 'Вопрос президенту' or tag == 'Война' or tag == 'Власть' or tag == 'Закон' or tag == 'Прямая линия' or tag == 'Жизнь' or tag == 'Актуально' or tag == 'Выборы' or tag == 'Государство' or tag == 'Армия' or tag == 'Владимир Путин' or tag == 'Криминал' or tag == 'ЛГБТ' or tag == 'Дискриминация' or tag == 'Ислам' or tag == 'История' or tag == 'Религия' or tag == 'Социум' or tag == 'Христианство' or tag == 'мероприятия' or tag == 'Социология' or tag == 'Москва' or tag == 'Политика' or tag == 'Общество' or tag == 'СССР' or tag == 'Экономика' or tag == 'Экология' or tag == 'Терроризм' or tag == 'Телевидение' or tag == 'СМИ' or tag == 'Феминизм':
            return 'Общество и история'
        if tag == 'Литература' or tag == 'ЕГЭ' or tag == 'Математика' or tag == 'Физика' or tag == 'Русский язык' or tag == 'Химия':
            return 'Помощь с учёбой'
        if tag == 'Города' or tag == 'Европа' or tag == 'Санкт-Петербург' or tag == 'Путешествия' or tag == 'США' or tag == 'Украина' or tag == 'Туризм' or tag == 'Россия' or tag == 'Страны':
            return 'Путешествия'
        if tag == 'Выбор профессии' or tag == 'Карьера' or tag == 'Программирование' or tag == 'Юриспруденция' or tag == 'Работа' or tag == 'Профессия':
            return 'Работа и карьера'
        if tag == 'Поэзия' or tag == 'Интервью' or tag == 'Космос' or tag == 'Интересное' or tag == 'Кино' or tag == 'Мемы' or tag == 'Рэп' or tag == 'смешное' or tag == 'Юмор' or tag == 'Сериалы' or tag == 'Сон' or tag == 'Музыка' or tag == 'Фотография':
            return 'Развлечения и досуг'
        if tag == 'Психика' or tag == 'Дружба' or tag == 'Девушки' or tag == 'Детский вопрос' or tag == 'Личность' or tag == 'Общение' or tag == 'Родители' or tag == 'Психотерапия' or tag == 'Психиатрия' or tag == 'Отношения' or tag == 'Женщины' or tag == 'Дети' or tag == 'Секс' or tag == 'Семья' or tag == 'Чувства' or tag == 'Любовь' or tag == 'Личный опыт' or tag == 'Мужчины':
            return 'Семья и отношения'
        if tag == 'Футбол' or tag == 'Спорт':
            return 'Спорт'
        if tag == 'Высшее образование' or tag == 'Маркетинг' or tag == 'Самосовершенствование' or tag == 'ВУЗ' or tag == 'Анатомия' or tag == 'Английский язык' or tag == 'География' or tag == 'Иностранные языки' or tag == 'Журналистика' or tag == 'Мозг' or tag == 'Саморазвитие' or tag == 'Теория науки' or tag == 'Психология' or tag == 'Лингвистика' or tag =='Книги' or tag =='Биология' or tag == 'Образование' or tag == 'Обучение' or tag == 'Школа' or tag == 'Язык' or tag == 'Философия' or tag == 'Что почитать?' or tag == 'Университет' or tag == 'Наука' or tag == 'Учеба':
            return 'Образование'
        if tag == 'Вопрос' or tag == 'Мифы' or tag == 'Сверхъестественное' or tag == 'Будущее' or tag == 'Лайфхак' or tag == 'Дом' or tag == 'Главная страница' or tag == 'TheQuestion' or tag == 'Классная история' or tag == 'Зашкваршн' or tag == 'Зачем я это узнал?' or tag == 'Быт' or tag == 'Архитектура' or tag == 'Темы нет' or tag == 'Что будет если' or tag == 'Этикет' or tag == 'Как это работает?' or tag == 'Урбанистика' or tag == 'Что будет, если' or tag == 'thequestion':
            return 'Другое'

In [50]:
questions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121937 entries, 0 to 0
Data columns (total 21 columns):
added            121937 non-null object
answersCount     121937 non-null int64
arating          121937 non-null object
bestanswer       121937 non-null int64
is_expert        121937 non-null int64
kpd              121937 non-null object
lvl              121937 non-null object
author_name      121937 non-null object
votes            121937 non-null object
qid              121937 non-null object
title            121937 non-null object
rating           121937 non-null object
tags             121937 non-null object
totalMarks       0 non-null object
usrid            121937 non-null object
tag              121937 non-null object
answered         121937 non-null int32
title_len        121937 non-null int64
service          121937 non-null object
author_gender    0 non-null float64
year_created     0 non-null float64
dtypes: float64(2), int32(1), int64(4), object(14)
memory usage: 20.0+ M

In [48]:
questions_df.rename(columns={'anscnt': 'answersCount', 'nick': 'author_name', 'qtext': 'title', 'points': 'votes'}, inplace=True)
questions_df['bestanswer'] = questions_df['bestanswer'].astype('int64')
questions_df['answersCount'] = questions_df['answersCount'].astype('int64')
questions_df['tags'] = questions_df['tags'].astype('str')
questions_df['title'] = questions_df['title'].astype('str')
questions_df['author_name'] = questions_df['author_name'].astype('str')


questions_df['tag'] = questions_df['tags'].apply(lambda x: tags_mapper('mailru', x))
questions_df['answered'] = np.where((questions_df['answersCount'] > 0) & (questions_df['bestanswer'] > 0),1,0)
questions_df['title_len'] = questions_df['title'].astype(str).apply(len)
questions_df = questions_df[questions_df['tag'].notnull()]


In [51]:
questions_df.head()

,added,answersCount,arating,bestanswer,is_expert,kpd,lvl,author_name,votes,qid,...,rating,tags,totalMarks,usrid,tag,answered,title_len,service,author_gender,year_created
0,208991296,18,0,422097449,0,27.36,Искусственный Интеллект,НЕНАСТОЯЩИЙ,123287,81166135,...,0,Юмор,None,71890278,Развлечения и досуг,1,34,mailru,NaN,NaN
0,208941307,6,0,422097333,0,10.26,Мыслитель,Санек АтаманOFF,6307,81166183,...,0,Юмор,None,15690846,Развлечения и досуг,1,34,mailru,NaN,NaN
0,208922419,5,0,422099105,0,9.39,Мыслитель,М@rg@ret,5180,81166262,...,0,Юмор,None,12164623,Развлечения и досуг,1,27,mailru,NaN,NaN
0,208810655,8,0,422097888,0,20.53,Мудрец,SOLOTZE,14694,81166263,...,0,Юмор,None,31646113,Развлечения и досуг,1,116,mailru,NaN,NaN
0,208842086,9,0,422098134,0,11.79,Искусственный Интеллект,Андрей Абидов,139949,81166278,...,0,Юмор,None,16203168,Развлечения и досуг,1,32,mailru,NaN,NaN


In [49]:
questions_df['service'] = 'mailru'
questions_df['author_gender'] = np.nan
questions_df['year_created'] = np.nan
tmp = pd.read_csv('all.csv')
all_df = pd.concat([questions_df[['answersCount', 'tag', 'title', 'title_len', 'author_name', 'service', 'votes', 'year_created', 'author_gender', 'answered']], tmp])
all_df.to_csv(index=False, path_or_buf='all.csv')

c:\users\mr_to\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
